In [7]:
!pip install tensorflow opencv-python matplotlib

  Using cached opencv_python-4.8.0.74-cp37-abi3-win_amd64.whl (38.1 MB)


In [156]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [253]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer , Conv2D , Dense , MaxPooling2D , Input , Flatten
import tensorflow as tf

In [260]:
!pip3 install tensorflow

In [262]:
!pip3 install --upgrade tensorflow

In [271]:
tf.config.list_physical_devices('GPU')

[]

In [268]:
# avoid out of mem errors
gpus  = tf.config.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    print(gpu)
    tf.config.experimental.set_memory_growth(gpu,True)
        

[]


In [159]:
POS_PATH = os.path.join('data' , 'positive')
NEG_PATH = os.path.join('data' , 'negative')
ANC_PATH = os.path.join('data' , 'anchor')

In [160]:
os.mkdir(POS_PATH)
os.mkdir(NEG_PATH)
os.mkdir(ANC_PATH)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data\\positive'

In [10]:
!tar -xf lfw.tgz

In [26]:
 for directory in os.listdir('lfw'):
        for file in os.listdir(os.path.join('lfw',directory)):
            EX_PATH = os.path.join('lfw' , directory , file)
            NEW_PATH = os.path.join(NEG_PATH , file)
            os.replace(EX_PATH , NEW_PATH)

In [161]:
import uuid

In [162]:
cap  = cv2.VideoCapture(0)
while cap.isOpened():
    ret , frame = cap.read()
    
    # 250*250
    frame = frame[120:120+250,200:200+250,:]
    
    #anchor collection
    if cv2.waitKey(1) & 0XFF == ord('a'):
        #create unique file path
        imgname = os.path.join(ANC_PATH , '{}.jpg'.format(uuid.uuid1()))
        #write out anchor image
        cv2.imwrite(imgname,frame)
        
    #pos collection
    if cv2.waitKey(1) & 0XFF == ord('p'):
        #create unique file path
        imgname = os.path.join(POS_PATH , '{}.jpg'.format(uuid.uuid1()))
        #write out anchor image
        cv2.imwrite(imgname,frame)
        
    
    
    cv2.imshow('Image collection' , frame)
    
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

#release the webcam
cap.release()
cv2.destroyAllWindows()

In [163]:
anchor = tf.data.Dataset.list_files(ANC_PATH + '\*.jpg').take(150)
positive = tf.data.Dataset.list_files(POS_PATH + '\*.jpg').take(150)
negative = tf.data.Dataset.list_files(NEG_PATH + '\*.jpg').take(150)

In [164]:
dir_test = anchor.as_numpy_iterator()

In [165]:
dir_test.next()

b'data\\anchor\\f3e31c3e-3541-11ee-a08d-d8f88368ebcd.jpg'

In [166]:
def preprocess(file_path):
    
    #read in image from file path
    byte_img = tf.io.read_file(file_path)
    #load in the image
    img = tf.io.decode_jpeg(byte_img)
    
    #resizing to 100*100
    img = tf.image.resize(img,(100,100))
    #scale between 0 - 1
    img = img /255.0
    return img

In [167]:
img = preprocess('data\\anchor\\e30aa3db-3488-11ee-9086-d8f88368ebcd.jpg')

In [176]:
img.numpy()

array([[[0.89387256, 0.9409314 , 0.9252451 ],
        [0.90539217, 0.9377451 , 0.925     ],
        [0.90661764, 0.93308824, 0.90857846],
        ...,
        [0.34901962, 0.3254902 , 0.34117648],
        [0.35196078, 0.32843137, 0.34411764],
        [0.3529412 , 0.32941177, 0.34509805]],

       [[0.9051471 , 0.9458333 , 0.9411765 ],
        [0.9073529 , 0.94191176, 0.93014705],
        [0.90882355, 0.94411767, 0.9245098 ],
        ...,
        [0.31936276, 0.30269608, 0.31936276],
        [0.31446078, 0.2977941 , 0.31446078],
        [0.31470588, 0.29803923, 0.32058823]],

       [[0.9019608 , 0.9411765 , 0.9372549 ],
        [0.90588236, 0.94558823, 0.93897057],
        [0.90294117, 0.94558823, 0.9286765 ],
        ...,
        [0.32058823, 0.30980393, 0.33529413],
        [0.33700982, 0.3262255 , 0.35171568],
        [0.3254902 , 0.31470588, 0.34019607]],

       ...,

       [[0.89387256, 0.9409314 , 0.93308824],
        [0.89191175, 0.9375    , 0.93039215],
        [0.8840686 , 0

In [187]:
#label the dataset
positives = tf.data.Dataset.zip((anchor,positive,tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor,negative,tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [186]:
data

<_ShuffleDataset element_spec=(TensorSpec(shape=(100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>

In [178]:
smaples = data.as_numpy_iterator()

In [185]:
len(smaples.next()[0])

100

In [182]:
def preprocess_twin(input_img , validation_img,label):
    return(preprocess(input_img) , preprocess(validation_img) , label)

In [188]:
#build dataloader pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size = 1024)

In [189]:
smaples = data.as_numpy_iterator()

In [193]:
len(smaples.next()[0])

100

In [194]:
# training partition
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [195]:
smaples = train_data.as_numpy_iterator()

In [196]:
len(smaples.next()[2])

16

test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))

In [197]:
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [198]:
test_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(None, 100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [199]:
inp = Input(shape=(105,105,3) , name = 'input_image')
print(inp)

KerasTensor(type_spec=TensorSpec(shape=(None, 105, 105, 3), dtype=tf.float32, name='input_image'), name='input_image', description="created by layer 'input_image'")


In [200]:
c1 = Conv2D(64,(10,10) , activation='relu')(inp)
print(c1)

KerasTensor(type_spec=TensorSpec(shape=(None, 96, 96, 64), dtype=tf.float32, name=None), name='conv2d/Relu:0', description="created by layer 'conv2d'")


In [201]:
m1 = MaxPooling2D(64,(2,2),padding='same')(c1)
print(m1)

KerasTensor(type_spec=TensorSpec(shape=(None, 48, 48, 64), dtype=tf.float32, name=None), name='max_pooling2d/MaxPool:0', description="created by layer 'max_pooling2d'")


In [202]:
c2 = Conv2D(128,(7,7) , activation='relu')(m1)
m2 = MaxPooling2D(64,(2,2),padding='same')(c2)
c3 = Conv2D(128,(4,4) , activation='relu')(m2)
m3 = MaxPooling2D(64,(2,2),padding='same')(c3)    
c4 = Conv2D(128,(4,4) , activation='relu')(m3)
f1 = Flatten()(c4)
d1 = Dense(4096,activation='sigmoid')(f1)
print(d1)

KerasTensor(type_spec=TensorSpec(shape=(None, 4096), dtype=tf.float32, name=None), name='dense_4/Sigmoid:0', description="created by layer 'dense_4'")


In [203]:
mod  = Model(inputs = [inp], outputs =[d1] ,name ='embedding')

In [204]:
mod.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 105, 105, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 96, 96, 64)        19264     
                                                                 
 max_pooling2d (MaxPooling2  (None, 48, 48, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 42, 42, 128)       401536    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 21, 21, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 18, 18, 128)       26

In [205]:
def make_embedding():
    inp = Input(shape=(100,100,3) , name ='input_image')
    #first block
    c1 = Conv2D(64,(10,10) , activation='relu')(inp)
    m1 = MaxPooling2D(64,(2,2),padding='same')(c1)
    
    #second block
    c2 = Conv2D(128,(7,7) , activation='relu')(m1)
    m2 = MaxPooling2D(64,(2,2),padding='same')(c2)
    
    #3rd block
    c3 = Conv2D(128,(4,4) , activation='relu')(m2)
    m3 = MaxPooling2D(64,(2,2),padding='same')(c3)
    
    #final embedding block
    
    c4 = Conv2D(128,(4,4) , activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096,activation='sigmoid')(f1)
    
    return Model(inputs = [inp], outputs =[d1] ,name ='embedding')

In [206]:
embedding = make_embedding()

In [207]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 100, 100, 3)]     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 91, 91, 64)        19264     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 46, 46, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 40, 40, 128)       401536    
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 20, 20, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 17, 17, 128)       26

In [208]:
#siamese L1 distance class
class L1Dist(Layer):
    def __init__(self,**kwargs):
        super().__init__()
        
        
    def call(self,input_embedding,validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)
    

In [209]:
l1  = L1Dist()

In [210]:
l1

In [211]:
def make_siamese_model():
    
    #Anchor image input in network
    input_image = Input(name='input_img' , shape=(100,100,3))
    
    #validation image in network
    validation_image = Input(name='validation_img' ,shape=(100,100,3))
    
    # combine siame  distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image) , embedding(validation_image))
    
    #classification layer
    classifier = Dense(1,activation='sigmoid')(distances)
    
    return Model(inputs=[input_image,validation_image],outputs=classifier , name='SiameseNetwork')

In [213]:
siamese_model = make_siamese_model()

In [214]:
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_img (InputLayer)      [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 validation_img (InputLayer  [(None, 100, 100, 3)]        0         []                            
 )                                                                                                
                                                                                                  
 embedding (Functional)      (None, 4096)                 1982380   ['input_img[0][0]',           
                                                          8          'validation_img[0][0]']      
                                                                                     

In [215]:
binary_cross_loss = tf.losses.BinaryCrossentropy()

In [216]:
opt = tf.keras.optimizers.Adam(1e-4)

In [217]:
checkpoint_dir = './training_checkpounts'
checkpoint_prefix = os.path.join(checkpoint_dir , 'ckpt')
checkpoint = tf.train.Checkpoint(opt =opt , siamese_model = siamese_model)

In [218]:
test_batch = train_data.as_numpy_iterator()
batch1 = test_batch.next()

In [230]:
batch1[:2]

(array([[[[0.9512255 , 0.98651963, 0.9669118 ],
          [0.94289213, 0.97818625, 0.9585784 ],
          [0.94730395, 0.98259807, 0.9629902 ],
          ...,
          [0.89436275, 0.9051471 , 0.8865196 ],
          [0.89093137, 0.9105392 , 0.89485294],
          [0.8990196 , 0.9147059 , 0.9107843 ]],
 
         [[0.95955884, 0.98308825, 0.967402  ],
          [0.95      , 0.9823529 , 0.9637255 ],
          [0.9487745 , 0.98406863, 0.9625    ],
          ...,
          [0.88259804, 0.90392154, 0.88161767],
          [0.8857843 , 0.90857846, 0.8928922 ],
          [0.88529414, 0.90882355, 0.9009804 ]],
 
         [[0.9588235 , 0.9823529 , 0.96666664],
          [0.95906866, 0.98259807, 0.9669118 ],
          [0.94411767, 0.9794118 , 0.9519608 ],
          ...,
          [0.88039213, 0.9017157 , 0.8838235 ],
          [0.8843137 , 0.904902  , 0.8892157 ],
          [0.88504905, 0.90857846, 0.9007353 ]],
 
         ...,
 
         [[0.92941177, 0.9764706 , 0.9745098 ],
          [0.93014

In [231]:
siamese_model(batch1[:2],training = True)

<tf.Tensor: shape=(16, 1), dtype=float32, numpy=
array([[0.500834  ],
       [0.5011255 ],
       [0.5011568 ],
       [0.5007561 ],
       [0.5011027 ],
       [0.500109  ],
       [0.5003884 ],
       [0.50179505],
       [0.50180763],
       [0.50088847],
       [0.5004047 ],
       [0.5007289 ],
       [0.5004124 ],
       [0.49959904],
       [0.5012542 ],
       [0.5001573 ]], dtype=float32)>

In [234]:
@tf.function
def train_step(batch):
    
    with tf.GradientTape() as tape:
        
        X = batch[:2]
        
        y= batch[2]
        
        yhat = siamese_model(X,training = True)
        
        loss = binary_cross_loss(y,yhat)
        
    grad = tape.gradient(loss,siamese_model.trainable_variables)
    
    opt.apply_gradients(zip(grad,siamese_model.trainable_variables))
    
    return loss
        

In [250]:
def train(data,EPOCHS):
    for epoch in range (1,EPOCHS+1):
        print('\n Ep {}/{}'.format(epoch,EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        for idx,batch in enumerate(data):
#             print(batch)
            
            train_step(batch)
            progbar.update(idx+1)
        
        if epoch % 10 ==0:
            checkpoint.save(file_prefix=checkpoint_prefix)

In [251]:
EPOCHS = 5

In [272]:
train(train_data,EPOCHS)


 Ep 1/5


KeyboardInterrupt: 